<a href="https://colab.research.google.com/github/MATHKIMS/dongwoo_project/blob/main/%EA%B3%B5%EA%B0%84%EC%A0%95%EB%B3%B4%EB%B6%84%EC%84%9D2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 필요한 패키지 설치
!pip install folium
import folium
from folium import Marker
from folium import plugins
from folium import GeoJson
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import plotly.graph_objects as go

plt.rcParams['figure.dpi'] = 300

In [22]:
# 서울 스타벅스 지점 데이터 불러오기
# https://www.kaggle.com/datasets/sewonghwang/starbucks-seoul
df=pd.read_csv("Starbucks_Seoul.csv")

# 지역 구분을 위한 json 파일 불러오기
import json
geo=json.load(open("Seoul_Gu.json",encoding='cp949'))  #json 파일 불러오는법

# 데이터 샘플 확인
df.head()

,name,address,gu_name,latitude,longitude
0,GS타워,서울특별시 강남구 논현로 508 (역삼동),강남구,37.501859,127.037278
1,SSG마켓도곡R,"서울특별시 강남구 언주로30길 57, 타워팰리스Ⅱ F 지하1층 (도곡동)",강남구,37.490298,127.054895
2,W-Mall,서울특별시 금천구 디지털로 188 (가산동),금천구,37.477305,126.887691
3,가든파이브,서울특별시 송파구 충민로 10 (문정동) 가든파이브툴,송파구,37.478232,127.119370
4,가락본동,서울특별시 송파구 송파대로30길 13 (가락동),송파구,37.494895,127.118785


In [20]:
geo

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[127.012, 37.582],
      [127.015, 37.582],
      [127.016, 37.582],
      [127.017, 37.582],
      [127.017, 37.581],
      [127.017, 37.58],
      [127.018, 37.58],
      [127.018, 37.579],
      [127.018, 37.578],
      [127.019, 37.578],
      [127.02, 37.578],
      [127.019, 37.578],
      [127.02, 37.578],
      [127.02, 37.579],
      [127.021, 37.579],
      [127.022, 37.579],
      [127.023, 37.578],
      [127.023, 37.577],
      [127.023, 37.576],
      [127.023, 37.575],
      [127.023, 37.572],
      [127.018, 37.57],
      [127.015, 37.57],
      [127.001, 37.569],
      [126.997, 37.569],
      [126.99, 37.568],
      [126.976, 37.569],
      [126.969, 37.568],
      [126.967, 37.566],
      [126.954, 37.579],
      [126.957, 37.58],
      [126.958, 37.584],
      [126.958, 37.594],
      [126.958, 37.598],
      [126.953, 37.6],
      [126.954, 37.605

In [7]:
# 기본 지도 시각화 (서울의 위도, 경도 입력)

m = folium.Map(location=[37.541, 126.986], zoom_start=12)
m

In [8]:
# 지도 형태 변경
m = folium.Map(location=[37.541, 126.986], tiles='Stamen Toner', zoom_start=12)

# 원하는 좌표에 반경(radius) 표시 (남산)
folium.CircleMarker([37.5538, 126.9810],radius=50,
                    popup='Laurelhurst Park', color='#3246cc',
                    fill_color='#3246cc').add_to(m)

# 원하는 좌표에 포인트 표시 (남산)
folium.Marker([37.5538, 126.9810], popup='The Waterfront').add_to(m)

m

In [9]:
# 서울 지도에 스타벅스 지점 수 시각화

m = folium.Map([37.541, 126.986], zoom_start=12 ,width="%100", height="%100")
locations = list(zip(df.latitude, df.longitude)) #latitude 컬럼, longitude 컬럼의 값이 튜플로 나오고 리스트화
cluster = plugins.MarkerCluster(locations=locations,
               popups=df["name"].tolist())
m.add_child(cluster)
m

In [ ]:
locations = list(zip(df.latitude, df.longitude))
locations

In [11]:
# 서울 지도에 스타벅스 지점수 도드맵 시각화

m = folium.Map(location=[37.541, 126.986], zoom_start=12, width="%100", height="%100")
locations = list(zip(df.latitude, df.longitude))
for i in range(len(locations)):
    folium.CircleMarker(location=locations[i],radius=1).add_to(m)
m

In [12]:
# 서울 구 별 스타벅스 지점 수 집계 및 중심점 산출
df_m = df.groupby('gu_name').agg({'latitude':'mean',
                                  'longitude':'mean',
                                  'name':'count'}).reset_index()
df_m.head()

,gu_name,latitude,longitude,name
0,강남구,37.507603,127.044611,80
1,강동구,37.539914,127.137106,14
2,강북구,37.626866,127.026372,5
3,강서구,37.555716,126.841528,16
4,관악구,37.481759,126.944286,11


In [21]:
# -*- coding: utf-8 -*-
import folium
import pandas as pd

# 기본 지도 생성
m = folium.Map(location=[37.541, 126.986], tiles='Cartodb Positron',
               zoom_start=11, width="%100", height="%100")

# 구별 구분선, 색상 설정
folium.Choropleth(
    geo_data=geo, # 앞에서 불러온 json 파일 적용
    fill_color="gray"
).add_to(m)

# 버블맵 삽입
locations = list(zip(df_m.latitude, df_m.longitude))
for i in range(len(locations)):
    row = df_m.iloc[i]
    folium.CircleMarker(location=locations[i],
                        radius=float(row["name"]/2), # 버블 크기 설정
                        fill_color="blue"
                       ).add_to(m)

# 지도 출력
m


In [27]:
# 미국 실업률 정보의 코로플레스맵 시각화를 위한 데이터, json 불러오기
# https://www.kaggle.com/datasets/sewonghwang/us-unemployment
df2 =pd.read_csv("us_states_unemployment.csv")

# 주 별 경계 json 파일 불러오기
import json
us_geo = json.load(open('folium_us-states.json',encoding='cp949'))

df2.head()

,State,Unemployment
0,AL,7.1
1,AK,6.8
2,AZ,8.1
3,AR,7.2
4,CA,10.1


In [28]:
# 미국 주별 실업률 코로플레스맵 시각화

# 미국 지도 시각화
m = folium.Map(location=[40,-98], zoom_start=3, tiles="Cartodb Positron")

# 지도에 주 경계선, 실업률 데이터 연동
m.choropleth(geo_data = us_geo, # json 데이터
               data = df2, # 실업률 데이터
               columns = ['State','Unemployment'], # 연동할 컬럼 설정
               key_on = 'feature.id', # json과 실업률 데이터를 연결할 키값 설정
               fill_color='YlGn',
               legend_name='실업률')

m

/usr/local/lib/python3.10/dist-packages/folium/folium.py:465: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(


In [30]:
# 서울과 각국의 수도 간의 커넥션맵 시각화

# 서울과 도쿄, 워싱턴, 마닐라, 파리, 모스크바 위경도 입력
source_to_dest = zip([37.541,37.541,37.541,37.541,37.541],
                     [35.6804, 38.9072, 14.5995, 48.8566,55.7558],
                     [126.986,126.986,126.986,126.986,126.986],
                     [139.7690, -77.0369, 120.9842, 2.3522,37.6173])

fig = go.Figure()

## for 문을 활용하여 위경도 입력
for a, b, c, d in source_to_dest:
    fig.add_trace(go.Scattergeo(
                        lat = [a, b],
                        lon = [c, d],
                        mode = 'lines',
                        line = dict(width = 1, color="red"),
                        opacity = 0.5 # 선 투명도
                        ))

fig.update_layout(
                margin={"t":0,"b":0,"l":0, "r":0, "pad":0},
                showlegend=False,
                geo = dict(
                showcountries=True) # 국가 경계선
                )

fig.show()